# 🤖 Análise de Consenso entre LLMs
## Notebook Refatorado com Alternative Params

Este notebook usa:
- Componentes modulares
- Logging com loguru
- Integração com HuggingFace
- **Alternative params** para testar variações

## 1️⃣ Setup e Configuração

In [1]:
# Imports
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from loguru import logger
import os

# Configurar logging
logger.remove()
logger.add(
    sys.stdout,
    format="<green>{time:HH:mm:ss}</green> | <level>{level: <8}</level> | <level>{message}</level>",
    level="INFO"
)

logger.success("✓ Setup completo")

17:21:30 | SUCCESS  | ✓ Setup completo


## 2️⃣ Carregar Dataset do HuggingFace

In [2]:
from src.utils.data_loader import load_hf_dataset, load_hf_dataset_as_dataframe, list_available_datasets

# Listar datasets
logger.info("Datasets disponíveis:")
for dataset in list_available_datasets():
    logger.info(f"  - {dataset}")

17:21:31 | Datasets disponíveis:
17:21:31 |   - agnews
17:21:31 |   - mpqa
17:21:31 |   - webkb
17:21:31 |   - ohsumed
17:21:31 |   - acm
17:21:31 |   - yelp_2013
17:21:31 |   - dblp
17:21:31 |   - books
17:21:31 |   - reut90
17:21:31 |   - wos11967
17:21:31 |   - twitter
17:21:31 |   - trec
17:21:31 |   - wos5736
17:21:31 |   - sst1
17:21:31 |   - pang_movie
17:21:31 |   - movie_review
17:21:31 |   - vader_movie
17:21:31 |   - subj
17:21:31 |   - sst2
17:21:31 |   - yelp_reviews
17:21:31 |   - 20ng
17:21:31 |   - medline


In [3]:
# Carregar dataset
dataset_name = "agnews"  # Ajuste conforme necessário

texts, categories, ground_truth = load_hf_dataset(dataset_name)

logger.info(f"Textos: {len(texts)}")
logger.info(f"Categorias: {categories}")
logger.info(f"Ground truth: {'Sim' if ground_truth else 'Não'}")

17:21:31 | Carregando dataset: agnews
17:21:31 | Combinando splits: ['train', 'test']
17:21:37 |   ✓ train: 510400 exemplos
17:21:43 |   ✓ test: 127600 exemplos
17:21:43 | Total combinado: 638000 exemplos
17:21:43 | Categorias extraídas automaticamente: [0, 1, 2, 3]
17:21:44 | Coluna de texto: text
17:21:44 | Ground truth carregado da coluna 'label'
17:21:44 | Textos: 638000
17:21:44 | Categorias: [0, 1, 2, 3]
17:21:44 | Ground truth: Sim


In [4]:
# Visualizar amostra
logger.info("\nAmostra dos textos:")
for i, text in enumerate(texts[:3]):
    logger.info(f"{i+1}. {text[:100]}...")
    if ground_truth:
        logger.info(f"   Label: {ground_truth[i]}")

17:21:44 | 
Amostra dos textos:
17:21:44 | 1. "Irish Claim 2nd Title","Notre Dame goalkeeper Erika Bohn seals the Irish's second NCAA championship...
17:21:44 |    Label: 1
17:21:44 | 2. "Court rules for BC in flap over exit fee","Boston College cleared a major legal hurdle in its bid t...
17:21:44 |    Label: 1
17:21:44 | 3. "Scientific Method Man","Gordon Rugg cracked the 400-year-old mystery of the Voynich manuscript. Nex...
17:21:44 |    Label: 3


In [5]:
df, categories = load_hf_dataset_as_dataframe(dataset_name)

17:21:44 | Carregando dataset: agnews
17:21:44 | Combinando splits: ['train', 'test']
17:21:50 |   ✓ train: 510400 exemplos
17:21:55 |   ✓ test: 127600 exemplos
17:21:55 | Total combinado: 638000 exemplos
17:21:55 | Categorias extraídas automaticamente: [0, 1, 2, 3]
17:21:56 | Coluna de texto: text
17:21:56 | Ground truth carregado da coluna 'label'
17:21:56 | DataFrame criado com 638000 linhas


In [6]:
df_test = df.iloc[df["label"] == 0]

NotImplementedError: iLocation based boolean indexing on an integer type is not available

In [ ]:
categories

[0, 1, 2, 3]

## 3️⃣ Configurar Modelos LLM

### Opção A: Usar apenas parâmetros padrão (temp=0)

In [ ]:
from src.llm_annotation_system.annotation.llm_annotator import LLMAnnotator
from src.experiments.base_experiment import DEFAULT_MODELS

# Inicializar SEM alternative params
annotator = LLMAnnotator(
    models=DEFAULT_MODELS,
    categories=categories,
    api_keys=None,
    use_langchain_cache=True,
    use_alternative_params=False  # Apenas temp=0
)

logger.success(f"✓ Annotator inicializado com {len(annotator.models)} modelos")

17:11:32 | Cache LangChain ativado: ..\..\data\.cache\hf\langchain_cache.db
17:11:32 | LLMAnnotator inicializado
17:11:32 | Modelos: 5 | Categorias: 4
17:11:32 | ✓ Annotator inicializado com 5 modelos


### Opção B: Usar alternative params (temp=0, 0.3, 0.5)

**Atenção**: Isso cria 9 modelos (3 base + 6 variações)

In [ ]:
# Descomente para usar alternative params:

# annotator = LLMAnnotator(
#     models=models,
#     categories=categories,
#     api_keys=None,
#     use_langchain_cache=True,
#     use_alternative_params=True  # Expande para 9 modelos
# )

# logger.success(f"✓ Annotator com alternative params: {len(annotator.models)} modelos")
# logger.info(f"  Modelos expandidos: {annotator.models}")

## 4️⃣ Executar Anotação

### Testar anotação

In [12]:
# Parâmetros
num_repetitions = 1
text = texts[0]
model = "llama3-8b"

# Estimativa
total_annotation = len(text) * num_repetitions
logger.warning(f"  Modelo: {model}")
logger.warning(f"  Texto: {len(text)}")
logger.warning(f"  Repetições: {num_repetitions}")

# Anotar
annotations = annotator.annotate_single(
    text=text,
    model=model,
    num_repetitions=num_repetitions
)

logger.success("✓ Anotação completa")

annotations

17:12:30 |   Modelo: llama3-8b
17:12:30 |   Texto: 191
17:12:30 |   Repetições: 1


first=ChatPromptTemplate(input_variables=['text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='You are an expert data annotator with extensive experience in text classification tasks.\n\nYour task is to classify the following text into one of the predefined categories with high precision.\n\n**Instructions:**\n1. Read the text carefully and understand its context\n2. Consider the nuances and implicit meanings\n3. Select the most appropriate category based on the content\n4. Be consistent with your classification criteria\n5. If the text is ambiguous, choose the most likely category based on dominant features\n\n**Available Categories:**\n- 0: 0\n- 1: 1\n- 2: 2\n- 3: 3\n\n**Text to classify:**\n{text}\n\n**Important Guidelines:**\n- Provide ONLY the category name as your response\n- Do not include explanations unless specifically requested\n- Be objective and avoid bias\n- Consider edge cases carefully\n- Maintain consistency across sim

17:12:35 | Erro em llama3-8b rep 1: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/chat (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000023044E26DD0>: Failed to establish a new connection: [WinError 10061] Nenhuma conexão pôde ser feita porque a máquina de destino as recusou ativamente'))
17:12:35 | ✓ Anotação completa


['ERROR']

### Anotando dataset completo

In [11]:
# Parâmetros
num_repetitions = 3

# Estimativa
total_annotations = len(texts) * len(annotator.models) * num_repetitions
logger.info(f"Total de anotações: {total_annotations}")
logger.warning(f"  Modelos: {len(annotator.models)}")
logger.warning(f"  Textos: {len(texts)}")
logger.warning(f"  Repetições: {num_repetitions}")

# Anotar
df_annotations = annotator.annotate_dataset(
    texts=texts,
    num_repetitions=num_repetitions
)

logger.success("✓ Anotações completas")
display(df_annotations.head())

17:11:36 | Total de anotações: 1500
17:11:36 |   Modelos: 5
17:11:36 |   Textos: 100
17:11:36 |   Repetições: 3
17:11:36 | Iniciando anotação
17:11:36 | Textos: 100 | Modelos: 5 | Repetições: 3
17:11:36 | Total de anotações: 1500
Anotando:   0%|          | 0/100 [00:00<?, ?it/s]

first=ChatPromptTemplate(input_variables=['text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='You are an expert data annotator with extensive experience in text classification tasks.\n\nYour task is to classify the following text into one of the predefined categories with high precision.\n\n**Instructions:**\n1. Read the text carefully and understand its context\n2. Consider the nuances and implicit meanings\n3. Select the most appropriate category based on the content\n4. Be consistent with your classification criteria\n5. If the text is ambiguous, choose the most likely category based on dominant features\n\n**Available Categories:**\n- 0: 0\n- 1: 1\n- 2: 2\n- 3: 3\n\n**Text to classify:**\n{text}\n\n**Important Guidelines:**\n- Provide ONLY the category name as your response\n- Do not include explanations unless specifically requested\n- Be objective and avoid bias\n- Consider edge cases carefully\n- Maintain consistency across sim

17:11:40 | Erro em deepseek-r1-8b rep 1: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/chat (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000230421EFE90>: Failed to establish a new connection: [WinError 10061] Nenhuma conexão pôde ser feita porque a máquina de destino as recusou ativamente'))
17:11:44 | Erro em deepseek-r1-8b rep 2: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/chat (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000230421F5490>: Failed to establish a new connection: [WinError 10061] Nenhuma conexão pôde ser feita porque a máquina de destino as recusou ativamente'))
17:11:48 | Erro em deepseek-r1-8b rep 3: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/chat (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000230421F4390>: Failed to establish a new connectio

first=ChatPromptTemplate(input_variables=['text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='You are an expert data annotator with extensive experience in text classification tasks.\n\nYour task is to classify the following text into one of the predefined categories with high precision.\n\n**Instructions:**\n1. Read the text carefully and understand its context\n2. Consider the nuances and implicit meanings\n3. Select the most appropriate category based on the content\n4. Be consistent with your classification criteria\n5. If the text is ambiguous, choose the most likely category based on dominant features\n\n**Available Categories:**\n- 0: 0\n- 1: 1\n- 2: 2\n- 3: 3\n\n**Text to classify:**\n{text}\n\n**Important Guidelines:**\n- Provide ONLY the category name as your response\n- Do not include explanations unless specifically requested\n- Be objective and avoid bias\n- Consider edge cases carefully\n- Maintain consistency across sim

17:11:52 | Erro em qwen2.5-7b rep 1: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/chat (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000230421FEF50>: Failed to establish a new connection: [WinError 10061] Nenhuma conexão pôde ser feita porque a máquina de destino as recusou ativamente'))
17:11:56 | Erro em qwen2.5-7b rep 2: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/chat (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000230421FFC90>: Failed to establish a new connection: [WinError 10061] Nenhuma conexão pôde ser feita porque a máquina de destino as recusou ativamente'))
17:12:00 | Erro em qwen2.5-7b rep 3: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/chat (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000230421F5510>: Failed to establish a new connection: [WinError

first=ChatPromptTemplate(input_variables=['text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='You are an expert data annotator with extensive experience in text classification tasks.\n\nYour task is to classify the following text into one of the predefined categories with high precision.\n\n**Instructions:**\n1. Read the text carefully and understand its context\n2. Consider the nuances and implicit meanings\n3. Select the most appropriate category based on the content\n4. Be consistent with your classification criteria\n5. If the text is ambiguous, choose the most likely category based on dominant features\n\n**Available Categories:**\n- 0: 0\n- 1: 1\n- 2: 2\n- 3: 3\n\n**Text to classify:**\n{text}\n\n**Important Guidelines:**\n- Provide ONLY the category name as your response\n- Do not include explanations unless specifically requested\n- Be objective and avoid bias\n- Consider edge cases carefully\n- Maintain consistency across sim

17:12:04 | Erro em gemma2-9b rep 1: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/chat (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000230421EF250>: Failed to establish a new connection: [WinError 10061] Nenhuma conexão pôde ser feita porque a máquina de destino as recusou ativamente'))
17:12:08 | Erro em gemma2-9b rep 2: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/chat (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002304220CA50>: Failed to establish a new connection: [WinError 10061] Nenhuma conexão pôde ser feita porque a máquina de destino as recusou ativamente'))
17:12:12 | Erro em gemma2-9b rep 3: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/chat (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000230421F5650>: Failed to establish a new connection: [WinError 10

first=ChatPromptTemplate(input_variables=['text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='You are an expert data annotator with extensive experience in text classification tasks.\n\nYour task is to classify the following text into one of the predefined categories with high precision.\n\n**Instructions:**\n1. Read the text carefully and understand its context\n2. Consider the nuances and implicit meanings\n3. Select the most appropriate category based on the content\n4. Be consistent with your classification criteria\n5. If the text is ambiguous, choose the most likely category based on dominant features\n\n**Available Categories:**\n- 0: 0\n- 1: 1\n- 2: 2\n- 3: 3\n\n**Text to classify:**\n{text}\n\n**Important Guidelines:**\n- Provide ONLY the category name as your response\n- Do not include explanations unless specifically requested\n- Be objective and avoid bias\n- Consider edge cases carefully\n- Maintain consistency across sim

Anotando:   0%|          | 0/100 [00:40<?, ?it/s]


KeyboardInterrupt: 

## 5️⃣ Calcular Consenso

In [ ]:
# Calcular consenso
df_with_consensus = annotator.calculate_consensus(df_annotations)

# Estatísticas
logger.info("\n📊 Estatísticas de Consenso:")
logger.info(f"  Média: {df_with_consensus['consensus_score'].mean():.2%}")
logger.info(f"  Mediana: {df_with_consensus['consensus_score'].median():.2%}")
logger.info(f"  Desvio padrão: {df_with_consensus['consensus_score'].std():.2%}")

# Distribuição por nível
levels = df_with_consensus['consensus_level'].value_counts()
logger.info("\nDistribuição por nível:")
for level, count in levels.items():
    logger.info(f"  {level}: {count} ({count/len(df_with_consensus):.1%})")

In [ ]:
# Visualizações
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histograma
axes[0].hist(df_with_consensus['consensus_score'], bins=20, edgecolor='black')
axes[0].set_xlabel('Consensus Score')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribuição de Scores de Consenso')

# Barras por nível
levels.plot(kind='bar', ax=axes[1], color=['green', 'orange', 'red'])
axes[1].set_xlabel('Nível de Consenso')
axes[1].set_ylabel('Contagem')
axes[1].set_title('Casos por Nível de Consenso')
axes[1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

## 6️⃣ Análise de Alternative Params

**Nota**: Esta seção só funciona se `use_alternative_params=True`

In [ ]:
# Verificar se alternative params foi usado
if annotator.use_alternative_params:
    logger.info("📊 Analisando impacto dos alternative params...")
    
    # Agrupar por modelo base
    for base_model in models:
        # Encontrar variações deste modelo
        variations = [m for m in annotator.models if m.startswith(base_model)]
        
        logger.info(f"\n{base_model}:")
        
        for var in variations:
            if f'{var}_consensus_score' in df_with_consensus.columns:
                score = df_with_consensus[f'{var}_consensus_score'].mean()
                logger.info(f"  {var}: {score:.2%} consenso interno")
    
    # Comparar temperaturas
    fig, ax = plt.subplots(figsize=(12, 6))
    
    consensus_cols = [col for col in df_with_consensus.columns 
                     if '_consensus_score' in col and '_alt' in col or 
                     (col.replace('_consensus_score', '') in models)]
    
    if consensus_cols:
        means = [df_with_consensus[col].mean() for col in consensus_cols]
        labels = [col.replace('_consensus_score', '') for col in consensus_cols]
        
        ax.bar(labels, means)
        ax.set_ylabel('Consenso Interno Médio')
        ax.set_title('Consenso por Variação de Parâmetros')
        ax.tick_params(axis='x', rotation=45)
        plt.tight_layout()
        plt.show()
    
else:
    logger.warning("Alternative params não foi usado. Para análise detalhada, reinicialize com use_alternative_params=True")

## 7️⃣ Análise Detalhada de Consenso

In [ ]:
from consensus_analyzer_refactored import ConsensusAnalyzer

# Inicializar analyzer
analyzer = ConsensusAnalyzer(categories)

# Colunas de consenso
consensus_cols = [col for col in df_with_consensus.columns if '_consensus' in col and '_score' not in col]

logger.info(f"Analisando {len(consensus_cols)} anotadores")

# Gerar relatório
report = analyzer.generate_consensus_report(
    df=df_with_consensus,
    annotator_cols=consensus_cols,
    output_dir="./results"
)

logger.success("✓ Relatório gerado")

In [ ]:
# Métricas
logger.info("\n📊 Métricas de Concordância:")
logger.info(f"  Fleiss' Kappa: {report['fleiss_kappa']:.3f} ({report['fleiss_interpretation']})")

# Interpretação
kappa = report['fleiss_kappa']
if kappa > 0.8:
    logger.success("Concordância excelente!")
elif kappa > 0.6:
    logger.info("Concordância boa")
elif kappa > 0.4:
    logger.warning("Concordância moderada")
else:
    logger.warning("Concordância fraca")

In [ ]:
# Matriz de concordância
agreement_df = report['pairwise_agreement']

plt.figure(figsize=(12, 10))
sns.heatmap(agreement_df, annot=True, fmt='.2f', cmap='YlGnBu', cbar_kws={'label': 'Agreement'})
plt.title('Matriz de Concordância Par a Par')
plt.tight_layout()
plt.show()

In [ ]:
# Casos problemáticos
problematic = report.get('problematic_cases')
if problematic is not None and len(problematic) > 0:
    logger.warning(f"\n⚠️  {len(problematic)} casos problemáticos identificados")
    display(problematic.head())
else:
    logger.success("\n✓ Nenhum caso problemático identificado")

## 8️⃣ Validação com Ground Truth

In [ ]:
if ground_truth:
    from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
    
    # Adicionar ground truth
    df_with_consensus['ground_truth'] = ground_truth
    
    # Accuracy
    accuracy = accuracy_score(
        df_with_consensus['ground_truth'],
        df_with_consensus['most_common_annotation']
    )
    
    logger.success(f"\n🎯 Accuracy: {accuracy:.2%}")
    
    # Classification report
    logger.info("\nClassification Report:")
    print(classification_report(
        df_with_consensus['ground_truth'],
        df_with_consensus['most_common_annotation']
    ))
    
    # Confusion matrix
    cm = confusion_matrix(
        df_with_consensus['ground_truth'],
        df_with_consensus['most_common_annotation']
    )
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix vs Ground Truth')
    plt.tight_layout()
    plt.savefig('./results/confusion_vs_ground_truth.png', dpi=150)
    plt.show()
    
else:
    logger.info("\n⚠️  Ground truth não disponível - pulando validação")

## 9️⃣ Exportar Resultados

In [ ]:
import json
from pathlib import Path

# Criar diretório
results_dir = Path('./results/final')
results_dir.mkdir(parents=True, exist_ok=True)

# Salvar CSVs
df_with_consensus.to_csv(results_dir / 'dataset_anotado_completo.csv', index=False)
logger.info(f"✓ Salvos: {len(df_with_consensus)} registros")

# Alta confiança
high_conf = df_with_consensus[df_with_consensus['consensus_score'] >= 0.8]
high_conf.to_csv(results_dir / 'alta_confianca.csv', index=False)
logger.info(f"✓ Alta confiança: {len(high_conf)} registros")

# Necessita revisão
low_conf = df_with_consensus[df_with_consensus['consensus_score'] < 0.8]
low_conf.to_csv(results_dir / 'necessita_revisao.csv', index=False)
logger.info(f"✓ Necessita revisão: {len(low_conf)} registros")

# Sumário JSON
summary = {
    'dataset': {
        'name': dataset_name,
        'total_texts': len(texts),
        'categories': categories,
        'has_ground_truth': ground_truth is not None
    },
    'config': {
        'models': models,
        'total_models': len(annotator.models),
        'use_alternative_params': annotator.use_alternative_params,
        'num_repetitions': num_repetitions,
        'total_annotations': len(texts) * len(annotator.models) * num_repetitions
    },
    'results': {
        'consensus_mean': float(df_with_consensus['consensus_score'].mean()),
        'consensus_median': float(df_with_consensus['consensus_score'].median()),
        'high_consensus': int((df_with_consensus['consensus_level'] == 'high').sum()),
        'medium_consensus': int((df_with_consensus['consensus_level'] == 'medium').sum()),
        'low_consensus': int((df_with_consensus['consensus_level'] == 'low').sum()),
    },
    'metrics': {
        'fleiss_kappa': float(report['fleiss_kappa']),
        'fleiss_interpretation': report['fleiss_interpretation']
    }
}

if ground_truth:
    summary['validation'] = {
        'accuracy': float(accuracy)
    }

with open(results_dir / 'sumario_experimento.json', 'w') as f:
    json.dump(summary, f, indent=2)

logger.success("\n✓ Resultados exportados com sucesso!")

## 🔟 Resumo Final

In [ ]:
logger.info("\n" + "="*80)
logger.success("RESUMO DO EXPERIMENTO")
logger.info("="*80)

logger.info(f"\n📊 Dataset: {dataset_name}")
logger.info(f"  Textos: {len(texts)}")
logger.info(f"  Categorias: {len(categories)}")

logger.info(f"\n🤖 Configuração:")
logger.info(f"  Modelos base: {len(models)}")
logger.info(f"  Total modelos: {len(annotator.models)}")
logger.info(f"  Alternative params: {annotator.use_alternative_params}")
logger.info(f"  Repetições: {num_repetitions}")

logger.info(f"\n📈 Consenso:")
logger.info(f"  Média: {df_with_consensus['consensus_score'].mean():.2%}")
logger.info(f"  Fleiss' Kappa: {report['fleiss_kappa']:.3f} ({report['fleiss_interpretation']})")

if ground_truth:
    logger.info(f"\n🎯 Validação:")
    logger.info(f"  Accuracy: {accuracy:.2%}")

logger.info(f"\n📁 Arquivos gerados em: {results_dir}/")

cache_stats = annotator.get_cache_stats()
logger.info(f"\n💾 Cache: {cache_stats['total_entries']} entradas")

logger.success("\n✅ Análise completa!")